# Some statistics for models

## ResNet

Number of trainable parameters for each ResNet depth, IE. the number of FLOPs (Floating Point Operations), which directly corresponds to complexity.

In [1]:
import torch
from tqdm import tqdm

In [2]:
from models.resnet import ResNet, ResidualBlock
from models.clinical_bert import bio_clinical_BERT

bert = bio_clinical_BERT()

resnet18 = ResNet(12, 5, 18, ResidualBlock)
resnet34 = ResNet(12, 5, 34, ResidualBlock)

c:\Users\ssjsi\Documents\Bachelors-project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
resnet18_params = sum(p.numel() for p in resnet18.parameters() if p.requires_grad)
resnet34_params = sum(p.numel() for p in resnet34.parameters() if p.requires_grad)
print(f"Number of trainable parameters in ResNet18: {resnet18_params:,}")
print(f"Number of trainable parameters in ResNet34: {resnet34_params:,}")

Number of trainable parameters in ResNet18: 3,851,397
Number of trainable parameters in ResNet34: 7,225,733


In [4]:
from configs.ptb_xl_configs import Configs
from dataloader.ptb_xl import ptb_xl_data_generator
configs = Configs()
train_loader, val_loader, test_loader = ptb_xl_data_generator(configs, include_text=True)

Loading PTB-XL...
Finished loading PTB-XL in 0 minutes and 3 seconds.


In [5]:
from train_eval.trainer import Trainer
device = 'cuda' if torch.cuda.is_available() else 'cpu'
trainer = Trainer()

In [6]:
resnet18 = resnet18.to(device)
bert = bert.to(device)

In [7]:
from models.linear_projectors import TextLinearProjectionHead, EcgLinearProjectionHead
text_linear_head = TextLinearProjectionHead(768)
ecg_linear_head = EcgLinearProjectionHead(512)

In [8]:
for i, data in tqdm(enumerate(train_loader)):
    ecg, text, target = data
    ecg = ecg.to(device)
    #text = text.to(device)
    target = target.to(device)

    ecg_output = resnet18(ecg)
    #text_output = bert(text)

    ecg_output_proj = ecg_linear_head(ecg_output)
    text_output_proj = text_linear_head(text_output)

    print(text, ecg, target)

    break

0it [00:00, ?it/s]

Finished encoding text in 0 minutes and 0 seconds.


0it [00:08, ?it/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)